In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from model_mo2cap2 import mo2cap_ours
from ours_model import Ours_model
from tqdm import tqdm
import os

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = Ours_model(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		image = test_batch['image'].cuda()
		image_zoom = test_batch['image_zoom'].cuda()
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {
			'image':image,
			# 'image_zoom' : image_zoom,
			}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			# break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp852'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp852/0.pth',
 '/workspace/2d_to_3d/apps/exp852/1.pth',
 '/workspace/2d_to_3d/apps/exp852/2.pth',
 '/workspace/2d_to_3d/apps/exp852/3.pth',
 '/workspace/2d_to_3d/apps/exp852/4.pth',
 '/workspace/2d_to_3d/apps/exp852/5.pth',
 '/workspace/2d_to_3d/apps/exp852/6.pth',
 '/workspace/2d_to_3d/apps/exp852/7.pth',
 '/workspace/2d_to_3d/apps/exp852/8.pth',
 '/workspace/2d_to_3d/apps/exp852/9.pth',
 '/workspace/2d_to_3d/apps/exp852/best.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	if i<5:continue
	_path = f'/workspace/2d_to_3d/apps/eval/Ours_new_model_WO_depthconv852'
	_name = f'baseline852_{i}.json'
	if not os.path.exists(_path):
		os.makedirs(_path)

	main(model_path=p,output_path=os.path.join(_path,_name))

06:35:23 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 80.60664367675781, 'UpperStretching': 84.89159393310547, 'Talking': 52.51578903198242, 'Reacting': 70.2246322631836, 'Walking': 79.1558609008789, 'Patting': 84.98297119140625, 'LowerStretching': 77.16140747070312, 'Gaming': 65.73369598388672, 'Greeting': 46.149131774902344, 'Gesticuling': 58.6947021484375}, 'UpperBody': {'All': 59.91231918334961, 'UpperStretching': 66.64013671875, 'Talking': 46.73014450073242, 'Reacting': 61.192264556884766, 'Walking': 52.49333190917969, 'Patting': 57.54693603515625, 'LowerStretching': 46.71407699584961, 'Gaming': 51.33243179321289, 'Greeting': 39.199031829833984, 'Gesticuling': 52.756004333496094}, 'LowerBody': {'All': 101.30096435546875, 'UpperStretching': 103.14305877685547, 'Talking': 58.301429748535156, 'Reacting': 79.25701904296875, 'Walking': 105.81836700439453, 'Patting': 112.41900634765625, 'LowerStretching': 107.60872650146484, 'Gaming': 80.13496398925781, 'Greeting': 53.099239349365234, 'Gesticuling': 64.63339233398438}}

  7%|▋         | 401/5352 [01:03<11:36,  7.11it/s]

{'FullBody': {'All': 79.74791717529297, 'UpperStretching': 84.391357421875, 'Talking': 51.944732666015625, 'Reacting': 67.32942962646484, 'Walking': 78.60559844970703, 'Patting': 79.79730224609375, 'LowerStretching': 75.17823791503906, 'Gaming': 65.32765197753906, 'Greeting': 48.479373931884766, 'Gesticuling': 56.67496871948242}, 'UpperBody': {'All': 59.377864837646484, 'UpperStretching': 66.1772232055664, 'Talking': 45.51456832885742, 'Reacting': 57.96905517578125, 'Walking': 51.539127349853516, 'Patting': 55.73735046386719, 'LowerStretching': 46.13787841796875, 'Gaming': 52.383827209472656, 'Greeting': 40.53095245361328, 'Gesticuling': 52.05605697631836}, 'LowerBody': {'All': 100.11798095703125, 'UpperStretching': 102.6054916381836, 'Talking': 58.374908447265625, 'Reacting': 76.68980407714844, 'Walking': 105.67206573486328, 'Patting': 103.85726165771484, 'LowerStretching': 104.21859741210938, 'Gaming': 78.27146911621094, 'Greeting': 56.427791595458984, 'Gesticuling': 61.2938728332519

 11%|█         | 601/5352 [01:29<10:12,  7.75it/s]

...

 93%|█████████▎| 5001/5352 [11:15<00:50,  6.90it/s]

{'FullBody': {'All': 78.60920715332031, 'UpperStretching': 82.34420776367188, 'Walking': 73.00530242919922, 'LowerStretching': 75.78003692626953, 'Talking': 57.37038803100586, 'Greeting': 60.38043212890625, 'Patting': 79.27578735351562, 'Reacting': 73.17428588867188, 'Gaming': 70.52310943603516, 'Gesticuling': 66.98640441894531}, 'UpperBody': {'All': 58.03538131713867, 'UpperStretching': 64.5239028930664, 'Walking': 49.326087951660156, 'LowerStretching': 46.20624542236328, 'Talking': 42.39072799682617, 'Greeting': 47.583518981933594, 'Patting': 53.56467819213867, 'Reacting': 57.26103973388672, 'Gaming': 47.907142639160156, 'Gesticuling': 53.392738342285156}, 'LowerBody': {'All': 99.18304443359375, 'UpperStretching': 100.16450500488281, 'Walking': 96.68452453613281, 'LowerStretching': 105.35382080078125, 'Talking': 72.35004425048828, 'Greeting': 73.17735290527344, 'Patting': 104.98690032958984, 'Reacting': 89.0875244140625, 'Gaming': 93.13907623291016, 'Gesticuling': 80.58007049560547}}

 97%|█████████▋| 5201/5352 [11:42<00:21,  7.06it/s]

{'FullBody': {'All': 78.61567687988281, 'UpperStretching': 82.35379791259766, 'Walking': 72.92969512939453, 'LowerStretching': 75.85240936279297, 'Talking': 57.303104400634766, 'Greeting': 60.64427185058594, 'Patting': 79.28624725341797, 'Reacting': 73.03089904785156, 'Gaming': 70.48275756835938, 'Gesticuling': 66.87866973876953}, 'UpperBody': {'All': 58.04355239868164, 'UpperStretching': 64.5463638305664, 'Walking': 49.281158447265625, 'LowerStretching': 46.22752380371094, 'Talking': 42.36518859863281, 'Greeting': 47.750465393066406, 'Patting': 53.47883987426758, 'Reacting': 57.238067626953125, 'Gaming': 47.82107162475586, 'Gesticuling': 53.30311584472656}, 'LowerBody': {'All': 99.18782043457031, 'UpperStretching': 100.1612319946289, 'Walking': 96.57825469970703, 'LowerStretching': 105.477294921875, 'Talking': 72.24100494384766, 'Greeting': 73.5380630493164, 'Patting': 105.09363555908203, 'Reacting': 88.82372283935547, 'Gaming': 93.14443969726562, 'Gesticuling': 80.45421600341797}}


100%|██████████| 5352/5352 [12:02<00:00,  7.41it/s]
07:47:53 [INFO] Main: Saving evaluation results...
07:47:53 [INFO] Main: Done.
